In [21]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
base = BaseOverlay("base.bit")
import cv2
import time

In [22]:
Mode = VideoMode(640,480,24)
hdmi_out = base.video.hdmi_out
hdmi_out.configure(Mode,PIXEL_BGR)
hdmi_out.start()

In [23]:
frame_out_w = 1920
frame_out_h = 1080
frame_in_w = 640
frame_in_h = 480

In [24]:
videoIn = cv2.VideoCapture(0)
videoIn.set(cv2.CAP_PROP_FRAME_WIDTH, frame_in_w);
videoIn.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_in_h);
print("Capture device is open: " + str(videoIn.isOpened()))

Capture device is open: True


In [25]:
def getFaceBox(net, frame, conf_threshold=0.7):
    frameOpencvDnn = frame.copy()
    frameHeight = frameOpencvDnn.shape[0]
    frameWidth = frameOpencvDnn.shape[1]  
    blob = cv2.dnn.blobFromImage(frameOpencvDnn, 1.0, (300, 300), [104, 117, 123], True, False) 
    net.setInput(blob)
    detections = net.forward()  
    bboxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > conf_threshold:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            bboxes.append([x1, y1, x2, y2])  
            cv2.rectangle(frameOpencvDnn, (x1, y1), (x2, y2), (0, 255, 0), int(round(frameHeight / 150)),
                         8)  
    return frameOpencvDnn, bboxes

In [26]:
faceProto = "./AgeGender/opencv_face_detector.pbtxt"
faceModel = "./AgeGender/opencv_face_detector_uint8.pb"

ageProto = "./AgeGender/age_deploy.prototxt"
ageModel = "./AgeGender/age_net.caffemodel"

genderProto = "./AgeGender/gender_deploy.prototxt"
genderModel = "./AgeGender/gender_net.caffemodel"

In [27]:
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
ageList = ['(0-2)', '(4-6)', '(8-12)', '(15-20)', '(25-32)', '(38-43)', '(48-53)', '(60-100)']
genderList = ['Male', 'Female']
ageNet = cv2.dnn.readNet(ageModel, ageProto)
genderNet = cv2.dnn.readNet(genderModel, genderProto)
faceNet = cv2.dnn.readNet(faceModel, faceProto)

In [28]:
padding = 20
cv2.waitKey(1)
t = time.time()
hasFrame, frame = videoIn.read()
frame = cv2.flip(frame, 1)
frameFace, bboxes = getFaceBox(faceNet, frame)
print(frame)

[[[203 188 175]
  [204 189 176]
  [207 193 178]
  ...
  [118 119 118]
  [119 119 119]
  [118 118 118]]

 [[204 191 173]
  [204 191 173]
  [205 191 176]
  ...
  [116 119 119]
  [118 119 119]
  [119 120 120]]

 [[203 193 174]
  [203 193 174]
  [201 190 174]
  ...
  [117 120 120]
  [117 120 120]
  [117 120 120]]

 ...

 [[103  62  48]
  [107  66  52]
  [122  78  65]
  ...
  [255 253 244]
  [255 249 235]
  [250 235 217]]

 [[115  77  60]
  [101  63  46]
  [107  69  52]
  ...
  [255 253 249]
  [255 251 246]
  [252 245 237]]

 [[201 158 142]
  [172 129 113]
  [181 138 122]
  ...
  [254 254 253]
  [253 253 251]
  [251 252 247]]]


In [29]:
for bbox in bboxes: 
    face = frame[max(0, bbox[1] - padding):min(bbox[3] + padding, frame.shape[0] - 1),
           max(0, bbox[0] - padding):min(bbox[2] + padding, frame.shape[1] - 1)]
    print("=======", type(face), face.shape) 
    blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
    print("======", type(blob), blob.shape)  
    genderNet.setInput(blob)   
    genderPreds = genderNet.forward()   
    print("++++++", type(genderPreds), genderPreds.shape, genderPreds)   
    gender = genderList[genderPreds[0].argmax()]  
    print("Gender : {}, conf = {:.3f}".format(gender, genderPreds[0].max()))
    ageNet.setInput(blob)
    agePreds = ageNet.forward()
    age = ageList[agePreds[0].argmax()]
    print(agePreds[0].argmax())  
    print("*********", agePreds[0])  
    print("Age Output : {}".format(agePreds))
    print("Age : {}, conf = {:.3f}".format(age, agePreds[0].max()))
    label = "{},{}".format(gender, age)
    cv2.putText(frameFace, label, (bbox[0], bbox[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2,
               cv2.LINE_AA)  
outframe = hdmi_out.newframe()
outframe[0:480,0:640,:] = frameFace[0:480,0:640,:]
img = hdmi_out.writeframe(outframe)
print(img)

======= <class 'numpy.ndarray'> (338, 278, 3)
====== <class 'numpy.ndarray'> (1, 3, 227, 227)
++++++ <class 'numpy.ndarray'> (1, 2) [[9.9986279e-01 1.3714161e-04]]
Gender : Male, conf = 1.000
3
********* [1.1687631e-05 5.1676147e-05 3.1144719e-03 9.7279638e-01 2.2140628e-02
 1.7134638e-03 1.4744374e-04 2.4184817e-05]
Age Output : [[1.1687631e-05 5.1676147e-05 3.1144719e-03 9.7279638e-01 2.2140628e-02
  1.7134638e-03 1.4744374e-04 2.4184817e-05]]
Age : (15-20), conf = 0.973
None


In [20]:
videoIn.release()
hdmi_out.stop()
del hdmi_out